In [1]:
import os
from dotenv import load_dotenv
import nest_asyncio
nest_asyncio.apply()
load_dotenv()

True

In [2]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')


## 1. Define a Simple Tool

In [4]:
from llama_index.core.tools import FunctionTool

def add(x: int, y: int) -> int:
    """Adds two integers together."""
    return x + y

def mystery(x: int, y: int) -> int: 
    """Mystery function that operates on top of two numbers."""
    return (x + y) * (x + y)

add_tool=FunctionTool.from_defaults(fn=add)
mystery_tool=FunctionTool.from_defaults(fn=mystery)


In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
response = llm.predict_and_call(
    [add_tool, mystery_tool], 
    "Tell me the output of the mystery function on 2 and 9", 
    verbose=True
)
print(str(response))

=== Calling Function ===
Calling function: mystery with args: {"x": 2, "y": 9}
=== Function Output ===
121
121


## 2. Define an Auto-Retrieval Tool

In [9]:
from llama_index.core import SimpleDirectoryReader
# load documents
file_path=["./2_ Multidoc_Agentic_RAG/data/metagpt.pdf"]
documents = SimpleDirectoryReader(input_files=file_path).load_data()

In [10]:
from llama_index.core.node_parser import SentenceSplitter
splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [11]:
print(nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: 2_ Multidoc_Agentic_RAG/data/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-24
last_modified_date: 2024-05-24

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to 

In [18]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex(nodes)
query_engine = vector_index.as_query_engine(similarity_top_k=2)

In [19]:
response = query_engine.query(
    "What are some high-level results of MetaGPT?", 
)
print(str(response))

MetaGPT achieves an average score of 3.9, surpassing ChatDev's score of 2.1. It simplifies the process of transforming abstract requirements into detailed class and function designs through a specialized division of labor and SOPs workflow. Additionally, MetaGPT's structured messaging and feedback mechanisms reduce loss of communication information and improve the execution of code.


In [20]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '23', 'file_name': 'metagpt.pdf', 'file_path': '2_ Multidoc_Agentic_RAG/data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-24', 'last_modified_date': '2024-05-24'}
{'page_label': '9', 'file_name': 'metagpt.pdf', 'file_path': '2_ Multidoc_Agentic_RAG/data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-24', 'last_modified_date': '2024-05-24'}


In [25]:
from llama_index.core.vector_stores import MetadataFilters

query_engine = vector_index.as_query_engine(
    similarity_top_k=2,
    filters=MetadataFilters.from_dicts(
        [
            {"key": "page_label", "value": "2"}
        ]
    )
)


In [26]:
response = query_engine.query(
    "What are some high-level results of MetaGPT?", 
)
print(str(response))

Some high-level results of MetaGPT include achieving a new state-of-the-art in code generation benchmarks with high Pass@1 scores, outperforming other popular frameworks in creating complex software projects, handling higher levels of software complexity effectively, offering extensive functionality, and demonstrating robustness and efficiency in task completion.


In [17]:
for n in response.source_nodes:
    print(n.metadata)

{'page_label': '2', 'file_name': 'metagpt.pdf', 'file_path': '2_ Multidoc_Agentic_RAG/data/metagpt.pdf', 'file_type': 'application/pdf', 'file_size': 16911937, 'creation_date': '2024-05-24', 'last_modified_date': '2024-05-24'}
